In [18]:
import pandas as pd 

In [19]:
columns = ["airline_sentiment", "airline_sentiment_confidence", "airline", "name", "text", "tweet_created"]
df = pd.read_csv("Tweets.csv", usecols=columns, )

In [20]:
df

,airline_sentiment,airline_sentiment_confidence,airline,name,text,tweet_created
0,neutral,1.0000,Virgin America,cairdin,@VirginAmerica What @dhepburn said.,2015-02-24 11:35:52 -0800
1,positive,0.3486,Virgin America,jnardino,@VirginAmerica plus you've added commercials t...,2015-02-24 11:15:59 -0800
2,neutral,0.6837,Virgin America,yvonnalynn,@VirginAmerica I didn't today... Must mean I n...,2015-02-24 11:15:48 -0800
3,negative,1.0000,Virgin America,jnardino,@VirginAmerica it's really aggressive to blast...,2015-02-24 11:15:36 -0800
4,negative,1.0000,Virgin America,jnardino,@VirginAmerica and it's a really big bad thing...,2015-02-24 11:14:45 -0800
...,...,...,...,...,...,...
14635,positive,0.3487,American,KristenReenders,@AmericanAir thank you we got on a different f...,2015-02-22 12:01:01 -0800
14636,negative,1.0000,American,itsropes,@AmericanAir leaving over 20 minutes Late Flig...,2015-02-22 11:59:46 -0800
14637,neutral,1.0000,American,sanyabun,@AmericanAir Please bring American Airlines to...,2015-02-22 11:59:15 -0800
14638,negative,1.0000,American,SraJackson,"@AmericanAir you have my money, you change my ...",2015-02-22 11:59:02 -0800


In [3]:
from sklearn.preprocessing import LabelEncoder

In [4]:
lb = LabelEncoder()
df["target"] = lb.fit_transform(df["airline_sentiment"])
df.drop("airline_sentiment", axis=1, inplace=True)

In [5]:
df["target"].value_counts() / len(df)

0    0.626913
1    0.211680
2    0.161407
Name: target, dtype: float64

In [6]:
vc = df["name"].value_counts()
df["user_frequency"] = df["name"].apply(lambda x: vc[x])

In [7]:
df["dow"] = pd.to_datetime(df["tweet_created"]).dt.dayofweek
df["month"] = pd.to_datetime(df["tweet_created"]).dt.month

In [8]:
df["airline"].value_counts()

United            3822
US Airways        2913
American          2759
Southwest         2420
Delta             2222
Virgin America     504
Name: airline, dtype: int64

In [9]:
df = pd.concat([df, pd.get_dummies(df["airline"])], axis=1)

In [10]:
features = df.columns[6:]
X = df[features]
y = df["target"]

In [11]:
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y)

In [13]:
rfc = RandomForestClassifier()
cv = cross_val_score(rfc, X_train, y_train, scoring="f1_macro")

In [14]:
cv.mean(), cv.std()

(0.4116990712646519, 0.008133138680388292)

In [15]:
import spacy
import en_core_web_sm
from spacy.lang.en.stop_words import STOP_WORDS

nlp = en_core_web_sm.load()

In [16]:
df['clean_text'] = df['text'].str.replace(r"[!\"#$%&()*+,-./:;<=>?@[\\\]^_`{|}~]+", " ") 

/tmp/ipykernel_322/804233554.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['clean_text'] = df['text'].str.replace(r"[!\"#$%&()*+,-./:;<=>?@[\\\]^_`{|}~]+", " ")


In [17]:
tokenized_doc = df['clean_text'].fillna('').apply(lambda x: nlp(x))
tokenized_doc = tokenized_doc.apply(lambda x: [token.lemma_ for token in x if token.text not in STOP_WORDS])